In [1]:
# Тюнинг
import optuna as opt
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_val_score, train_test_split, StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.cluster import KMeans

# Пайплайн
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin, BaseEstimator

# Данные
import os
import pandas as pd
import numpy as np
from category_encoders import BinaryEncoder

In [2]:
# Пути
ROOT = os.getcwd()
TRAIN_DATASET = os.path.join(ROOT, '../data/train_AIC.csv')
TEST_DATASET = os.path.join(ROOT, '../data/test_AIC.csv')

# Загрузка
train_df = pd.read_csv(TRAIN_DATASET)
test_df = pd.read_csv(TEST_DATASET)

X, y = train_df.iloc[:, :-1], train_df.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
X_train = X_train.drop([
    'Категорийный менеджер', 'Изменение позиции заказа на закупку: изменение даты поставки на бумаге',
    'Количество', 'Дней между 0_1', 'Дней между 1_2', 'Дней между 2_3', 'Дней между 3_4', 'Дней между 4_5', 
    'Дней между 5_6', 'Дней между 6_7', 'Дней между 7_8', 'Согласование заказа 1', 'Согласование заказа 2',
    'Согласование заказа 3', 'Изменение даты поставки 7', 'Изменение даты поставки 15', 'Изменение даты поставки 30'
    ], axis=1)
X_test = X_test.drop([
    'Категорийный менеджер', 'Изменение позиции заказа на закупку: изменение даты поставки на бумаге',
    'Количество', 'Дней между 0_1', 'Дней между 1_2', 'Дней между 2_3', 'Дней между 3_4', 'Дней между 4_5', 
    'Дней между 5_6', 'Дней между 6_7', 'Дней между 7_8', 'Согласование заказа 1', 'Согласование заказа 2',
    'Согласование заказа 3', 'Изменение даты поставки 7', 'Изменение даты поставки 15', 'Изменение даты поставки 30'
    ], axis=1)

In [3]:
сat_features = [
    'Поставщик', 'Материал', 'Категорийный менеджер', 'Операционный менеджер',
    'Завод', 'Закупочная организация', 'Группа закупок', 'Балансовая единица',
    'ЕИ', 'Вариант поставки'
    ]

In [6]:
# Препроцессоры
class DataPreprocessor(BaseEstimator, TransformerMixin):
    """ Предобработчик данных """
    def __init__(self, transform_train=True):
        global cat_features
        self.transform_train = transform_train
        self.bin_encoder = BinaryEncoder(cols=cat_features)

    def fit(self, X, y=None):
        self.bin_encoder.fit(X)
        return self
    
    def transform(self, X):
        # Создаём копию датасета
        X_ = X.copy()

        # Категориальные фичи
        X_ = self.bin_encoder.transform(X_)

        # Временные фичи
        X_['day_sin'] = np.sin(np.pi * 2 * X_['День недели 2'] / 6)
        X_['day_cos'] = np.cos(np.pi * 2 * X_['День недели 2'] / 6)

        # Нумерация фич для LGBM (не принимает JSON символы)
        X_.columns = [num for num in range(0, len(X_.columns))]

        return X_


In [7]:
data_preprocessor = DataPreprocessor()
X_train_preproc = data_preprocessor.transform(X_train)
X_test_preproc = data_preprocessor.transform(X_test)

NameError: name 'cat_features' is not defined

In [66]:
# Функция оптимизации
def objective_lgbm(trial: opt.Trial):
    # Параметры
    learning_rate = trial.suggest_float('learning_rate', 0.1, 1, log=True)
    n_estimators = trial.suggest_int('n_estimators', 300, 1500, 50)
    max_depth = trial.suggest_int('max_depth', 6, 24)
    max_bin = trial.suggest_int('max_bin', 64, 256),
    num_leaves = trial.suggest_int('num_leaves', 64, 512)
    reg_lambda = trial.suggest_float('l2_reg', 0.1, 1.5, log=True)

    # Модель
    model = LGBMClassifier(
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        max_depth=max_depth,
        reg_lambda=reg_lambda,
        max_bin=max_bin,
        n_jobs=-1,
        force_col_wise=True,
        verbose=-1
        )

    model.fit(X_train_preproc, y_train)
    # cv_score = cross_val_score(model, X_train_lgbm, y_train, cv=StratifiedKFold(), scoring='f1_macro', n_jobs=-1)
    # accuracy = cv_score.mean()
    accuracy = f1_score(y_test, model.predict(X_test_preproc), average='macro')
    return accuracy

In [68]:
X_train_lgbm

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
187714,231,27439,5,1,1,9,1,1,1,1,...,2,6.746477,5,7,8,8,0,3,1.0,0
41651,33,6056,2,1,1,3,1,1,17,1,...,2,5.304034,26,7,7,7,0,0,1.0,10
102561,629,3566,2,1,1,36,1,5,31,2,...,1,6.330550,3,6,6,6,0,0,1.0,11
31316,31,536,10,2,2,3,2,1,3,2,...,4,4.883392,37,6,6,7,0,1,1.0,17
113039,106,27439,25,1,1,76,1,6,1,1,...,2,9.557131,1,6,6,6,1,1,1.0,69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119879,308,27439,1,17,17,166,16,1,14,2,...,0,7.304557,23,4,4,4,0,1,0.0,6
103694,2,191,11,5,4,31,4,1,48,2,...,4,5.347076,11,6,6,7,0,0,1.0,0
131932,250,27439,5,1,1,85,1,1,1,1,...,5,8.368166,2,11,11,11,0,0,1.0,16
146867,38,686,2,1,1,5,1,1,5,2,...,1,7.083772,14,9,9,9,2,1,3.0,41


In [67]:
study = opt.create_study(direction='maximize')
study.optimize(objective_lgbm, n_trials=50)

[I 2023-08-07 14:11:27,771] A new study created in memory with name: no-name-97f23b36-e5c0-4272-a3ff-ace420135cf6
[I 2023-08-07 14:11:46,367] Trial 0 finished with value: 0.8678564836735655 and parameters: {'learning_rate': 0.17203021582004538, 'n_estimators': 900, 'max_depth': 22, 'max_bin': 204, 'num_leaves': 279, 'l2_reg': 0.9858348908327655}. Best is trial 0 with value: 0.8678564836735655.
[I 2023-08-07 14:11:58,532] Trial 1 finished with value: 0.8588782548181872 and parameters: {'learning_rate': 0.20942974124828925, 'n_estimators': 600, 'max_depth': 13, 'max_bin': 155, 'num_leaves': 323, 'l2_reg': 0.33324771393741603}. Best is trial 0 with value: 0.8678564836735655.
[I 2023-08-07 14:12:15,926] Trial 2 finished with value: 0.8785134817944429 and parameters: {'learning_rate': 0.40103786377805345, 'n_estimators': 1050, 'max_depth': 21, 'max_bin': 92, 'num_leaves': 308, 'l2_reg': 0.10088188686900121}. Best is trial 2 with value: 0.8785134817944429.
[I 2023-08-07 14:12:38,038] Trial 3

In [70]:
study.best_params

{'learning_rate': 0.5725847818837073,
 'n_estimators': 1350,
 'max_depth': 13,
 'max_bin': 147,
 'num_leaves': 508,
 'l2_reg': 0.9729227793947609}